In [ ]:
try:
  import mne
  import numpy as np
  import os
  import glob
  from zipfile import ZipFile
  import tensorflow as tf
  from sklearn.preprocessing import StandardScaler
  from sklearn.model_selection import train_test_split
  # drive.mount('/content/drive')
  from keras.models import Sequential
  from keras.layers import LSTM, Dense
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import StandardScaler

  from sklearn.preprocessing import LabelEncoder
  # from keras.utils import to_categorical

  from keras.models import Sequential
  from keras.layers import LSTM, GRU, Dense, Dropout, TimeDistributed, BatchNormalization, Flatten, Activation
  from keras import regularizers
  from keras.callbacks import EarlyStopping, ModelCheckpoint
  from keras.models import load_model
  import matplotlib.pyplot as plt

  from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

  import warnings
except ModuleNotFoundError:
  !pip install mne

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
work_dir = '/content/drive/My Drive/data_eeg'

Mounted at /content/drive


In [ ]:
#lstm loading and processing
model_path="/content/drive/MyDrive/data_eeg/data_eeg/model/BiLSTM_final_model_allsubs_latest_april.h5"

# Load the pre-trained model # Replace with your model path
pretrained_model = tf.keras.models.load_model(model_path)

# Print the summary of the model to inspect the input and output layers
pretrained_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 640, 100)          20800     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 640, 50)           25200     
 onal)                                                           
                                                                 
 lstm_2 (LSTM)               (None, 640, 12)           3024      
                                                                 
 flatten (Flatten)           (None, 7680)              0         
                                                                 
 dense (Dense)               (None, 50)                384050    
                                                                 
 batch_normalization (Batch  (None, 50)                2

In [ ]:
#@title Filtering
def apply_filter(file_path):

    raw = mne.io.read_raw_edf(file_path, preload=True)

    # Set channel names
    channels = ['Fp1', 'Fp2', 'F3', 'F4', 'O1', 'O2', 'Fz', 'Pz']
    raw.pick_channels(channels)
    raw.set_eeg_reference(ref_channels=['O1', 'O2'])
    freqs = np.arange(50, 126, 50)
    raw.notch_filter(freqs=freqs, picks='eeg')
    raw.filter(l_freq=1.0, h_freq=40);

    # raw.set_eeg_reference('average')
    # freqs = np.arange(50, 126, 50)
    # raw.notch_filter(freqs=freqs, picks='eeg')
    # raw.filter(l_freq=0.5, h_freq=100)

    return raw


In [ ]:
SUB_PATH="/content/drive/MyDrive/data_eeg/data_eeg"

In [ ]:
#@title ica and segmentation
def extract_think_epochs(file_path):
    # Load the raw EDF file
    raw = apply_filter(file_path)

    # Extract events and event dictionary from annotations
    events, event_dict = mne.events_from_annotations(raw)

    # Filter events to include only those corresponding to the 'think' annotation
    think_event_id = event_dict['think']
    think_events = events[events[:, 2] == think_event_id]

    # Create epochs using the filtered 'think' events
    epochs = mne.Epochs(raw, think_events, event_id={'think': think_event_id},
                        tmin=0, tmax=3, baseline=(0, 0), preload=True, event_repeated="merge")

    # Return the epoch data as a NumPy array
    return epochs.get_data()

# List to hold the epochs from all files
sub_epochs = []

# Iterate over files in the directory and process each EDF file

for foldername, _, filenames in os.walk(SUB_PATH):
    for filename in filenames:
        if filename.endswith(".edf"):
            file_path = os.path.join(foldername, filename).replace("\\", "/")
            print("Processing file:", file_path)
            # Extract think epochs from the current file and append to the list
            epochs_data = extract_think_epochs(file_path)
            sub_epochs.append(epochs_data)

# # # Concatenate all epochs into a single NumPy array
all_epochs_array = np.concatenate(sub_epochs, axis=0)
np.save('/content/drive/MyDrive/data_eeg/data_eeg/aearray',all_epochs_array)
# # # Check the shape of the resulting array
print("Shape of all epochs array:", all_epochs_array.shape)

Processing file: /content/drive/MyDrive/data_eeg/data_eeg/Sub-1/Exp5/sub1_exp5.edf
Extracting EDF parameters from /content/drive/MyDrive/data_eeg/data_eeg/Sub-1/Exp5/sub1_exp5.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 32249  =      0.000 ...   128.996 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter 

<ipython-input-10-470cf99e3fe6>:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return epochs.get_data()


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 32249  =      0.000 ...   128.996 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopb

<ipython-input-10-470cf99e3fe6>:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return epochs.get_data()


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 32249  =      0.000 ...   128.996 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopb

...

Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 32249  =      0.000 ...   128.996 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopb

<ipython-input-10-470cf99e3fe6>:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return epochs.get_data()


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 32249  =      0.000 ...   128.996 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopb

<ipython-input-10-470cf99e3fe6>:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return epochs.get_data()


Shape of all epochs array: (400, 8, 751)


In [ ]:
l=["LRUDRLDU","UDLRDURL","ULRDLDRU","LUDRURDL","DRLURLDU"]
y_values=[]
for i in range(1,11):
  for j in l:
    for s in j:
      if s=="L":
          y_values.append(0)
      elif s=="R":
          y_values.append(1)
      elif s=="U":
          y_values.append(2)
      elif s=="D":
          y_values.append(3)

y_val=np.array(y_values)
print(y_val.shape)
print(y_val)
np.save('/content/drive/MyDrive/data_eeg/data_eeg/ylabels',y_val)

(400,)
[0 1 2 3 1 0 3 2 2 3 0 1 3 2 1 0 2 0 1 3 0 3 1 2 0 2 3 1 2 1 3 0 3 1 0 2 1
 0 3 2 0 1 2 3 1 0 3 2 2 3 0 1 3 2 1 0 2 0 1 3 0 3 1 2 0 2 3 1 2 1 3 0 3 1
 0 2 1 0 3 2 0 1 2 3 1 0 3 2 2 3 0 1 3 2 1 0 2 0 1 3 0 3 1 2 0 2 3 1 2 1 3
 0 3 1 0 2 1 0 3 2 0 1 2 3 1 0 3 2 2 3 0 1 3 2 1 0 2 0 1 3 0 3 1 2 0 2 3 1
 2 1 3 0 3 1 0 2 1 0 3 2 0 1 2 3 1 0 3 2 2 3 0 1 3 2 1 0 2 0 1 3 0 3 1 2 0
 2 3 1 2 1 3 0 3 1 0 2 1 0 3 2 0 1 2 3 1 0 3 2 2 3 0 1 3 2 1 0 2 0 1 3 0 3
 1 2 0 2 3 1 2 1 3 0 3 1 0 2 1 0 3 2 0 1 2 3 1 0 3 2 2 3 0 1 3 2 1 0 2 0 1
 3 0 3 1 2 0 2 3 1 2 1 3 0 3 1 0 2 1 0 3 2 0 1 2 3 1 0 3 2 2 3 0 1 3 2 1 0
 2 0 1 3 0 3 1 2 0 2 3 1 2 1 3 0 3 1 0 2 1 0 3 2 0 1 2 3 1 0 3 2 2 3 0 1 3
 2 1 0 2 0 1 3 0 3 1 2 0 2 3 1 2 1 3 0 3 1 0 2 1 0 3 2 0 1 2 3 1 0 3 2 2 3
 0 1 3 2 1 0 2 0 1 3 0 3 1 2 0 2 3 1 2 1 3 0 3 1 0 2 1 0 3 2]


In [ ]:
#@title data loading
import numpy as np

# Load your new dataset
X_new = np.load('/content/drive/MyDrive/data_eeg/data_eeg/aearray.npy')  # Replace with your file path
Y_new = np.load('/content/drive/MyDrive/data_eeg/data_eeg/ylabels.npy')  # Replace with your file path

# Ensure X_new and y_new are correctly shaped
print(X_new.shape)  # Should print (400, 8, 751)
print(Y_new.shape)  # Ensure it matches the number of samples


(400, 8, 751)
(400,)


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load your dataset
# Assume X_new is your new dataset with shape (400, 8, 751) # Replace with actual labels

# Step 1: Normalize the data
X_reshaped = X_new.reshape(-1, 1)  # Reshape the data to be 2D for fitting
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reshaped)
X_scaled = X_scaled.reshape(X_new.shape)

# Step 2: Reshape the data
# Flatten each trial and channel to a 1D vector
X_processed = []
Y_processed = []

for i in range(X_scaled.shape[0]):
    trial_data = X_scaled[i]
    trial_label = Y_new[i]
    # Iterate through channels
    for channel_data in trial_data:
        X_processed.append(channel_data)
        Y_processed.append(trial_label)

# Convert lists to numpy arrays
X_processed = np.array(X_processed)
Y_processed = np.array(Y_processed)

# The shape of X_processed should now be (400 * 8, 751) -> (3200, 751)
# We need to further reshape it to (3200, 640, 1)
X_processed_reshaped = X_processed[:, :640]  # Take the first 640 timesteps
X_processed_reshaped = X_processed_reshaped.reshape(-1, 640, 1)

# Correctly repeat the labels
Y_processed_repeated = np.repeat(Y_new, 8)  # Repeat each label 8 times for 8 channels

# Verify the shapes
print(X_processed_reshaped.shape)  # Expected: (3200, 640, 1)
print(Y_processed_repeated.shape)  # Expected: (3200,)

# Split the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_processed_reshaped, Y_processed_repeated, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

print(X_train.shape, X_val.shape, X_test.shape)  # Verify the split sizes
print(y_train.shape, y_val.shape, y_test.shape)


(3200, 640, 1)
(3200,)
(1920, 640, 1) (640, 640, 1) (640, 640, 1)
(1920,) (640,) (640,)


In [ ]:
#@title transfer learning
# Define early stopping and model checkpoint
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='LSTM_model_finetuned.keras', monitor='val_accuracy', verbose=1, save_best_only=True)
callbacks_list = [early_stopping, checkpointer]


# Compile the model again to ensure it is ready for training
pretrained_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model on the new dataset
history = pretrained_model.fit(X_train, y_train,
                               batch_size=32, epochs=20, shuffle=False,
                               validation_data=(X_val, y_val),
                               callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = pretrained_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")


NameError: name 'tf' is not defined

In [ ]:
# LSTM

def build_LSTM(input_shape):
    model = Sequential()
    #1
    model.add(LSTM(50, return_sequences=True, stateful=False,
             dropout = 0.5, input_shape=input_shape))

    #2
    model.add(LSTM(25, return_sequences=True, stateful=False,
            dropout = 0.3))

    #3
    model.add(LSTM(12, return_sequences=True, stateful=False,
             dropout = 0.1))
    model.add(Flatten())

    #4
    model.add(Dense(50))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    #model.add(Dropout(0.5))

    #5
    model.add(Dense(4, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#     # Set up early stopping
#     early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

#     #define early stopping callback
#     earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=30, mode='auto')

#     # saves the model weights after each epoch if the validation loss decreased
#     checkpointer = ModelCheckpoint(filepath='/home/ssn/miniconda3/envs/innerspeech/Project/LSTM_model_gamma.hdf5', monitor='val_acc', verbose=1, save_best_only=True)

#     callbacks_list = [early_stopping, checkpointer]

    #model.summary()
    return model


In [ ]:
input_shape = (X_train.shape[1],1)
print(input_shape)

(640, 1)


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
# checkpointer = ModelCheckpoint(filepath="/content/drive/MyDrive/eeg_research/data_eeg/model/new_LSTM_model.keras", monitor='val_accuracy', verbose=1, save_best_only=True)
callbacks_list = [early_stopping, checkpointer]
model = build_LSTM(input_shape)
# model.load_weights("/content/drive/MyDrive/eeg_research/data_eeg/model/new_LSTM_model.keras")
# history = model.fit(X_train, y_train,
#                     batch_size=32, epochs=20, shuffle=False,
#                     callbacks=callbacks_list,
#                     validation_data = (X_val,y_val))
history = model.fit(X_train, y_train,
                    batch_size=32, epochs=20, shuffle=False,
                    validation_data = (X_val,y_val))

Epoch 1/20
60/60 [==============================] - 71s 942ms/step - loss: 1.4577 - accuracy: 0.2646 - val_loss: 1.4292 - val_accuracy: 0.2531
Epoch 2/20
60/60 [==============================] - 43s 713ms/step - loss: 1.3779 - accuracy: 0.2891 - val_loss: 1.4139 - val_accuracy: 0.2406
Epoch 3/20
60/60 [==============================] - 43s 717ms/step - loss: 1.3687 - accuracy: 0.2901 - val_loss: 1.4250 - val_accuracy: 0.2688
Epoch 4/20
60/60 [==============================] - 41s 683ms/step - loss: 1.3549 - accuracy: 0.3286 - val_loss: 1.4753 - val_accuracy: 0.2375
Epoch 5/20
60/60 [==============================] - 41s 690ms/step - loss: 1.3432 - accuracy: 0.3328 - val_loss: 1.4314 - val_accuracy: 0.2641
Epoch 6/20
60/60 [==============================] - 40s 676ms/step - loss: 1.3366 - accuracy: 0.3307 - val_loss: 1.4327 - val_accuracy: 0.2547
Epoch 7/20
60/60 [==============================] - 58s 968ms/step - loss: 1.3390 - accuracy: 0.3323 - val_loss: 1.4557 - val_accuracy: 0.2500

In [ ]:
# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()